# 1. 교차 검증과 그리드 서치 

- 일반화 성능을 높이려면 가능한 테스트 세트를 사용하면 안됨 
- 그렇다면 하이퍼파리미터 튜닝을 어떻게 할 것인가?

## 1) 검증 세트 
- 테스트 세트를 사용하지 않고 검증세트를 사용 
- 일반적으로 훈련 60% + 검증 20% + 테스트 20% 
- 훈련 세트로 모델 훈련  후 검증 세트로 모델을 평가 > 테스트 하고 싶은 매개 변수를 바꿔가며 가장 좋은 모델을 고름 

## 2) 검증세트 시도

In [3]:
import pandas as pd

wine = pd.read_csv("https://bit.ly/wine_csv_data")
data = wine[["alcohol","sugar","pH"]]
target = wine["class"]

- 훈련 세트 나누기

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input , train_target, test_target = train_test_split(data,target,test_size=0.2, random_state=42 )

### 검증 세트 나누기 
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(val_input.shape, test_input.shape)

(1040, 3) (1300, 3)


## 3) 학습 및 평가
  - 과대적합 모델 

In [5]:
from sklearn.tree import DecisionTreeClassifier
# 결정 트리 모델 생성
dt = DecisionTreeClassifier(random_state=42)
# 모델 학습
dt.fit(sub_input, sub_target)

# 검증 세트로 모델 평가
print("훈련 세트 : ",dt.score(sub_input, sub_target))
print("검증 세트 : " , dt.score(val_input, val_target))

훈련 세트 :  0.9971133028626413
검증 세트 :  0.864423076923077


## 4) 교차 검증 
  - 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복함 
- 점수를 평균하여 최종 검증 점수를 얻음
- ex) 3-폴드 교차 검증 : 훈련 세트를 3개로 나누어 3번 반복함 (k겹 교차 검증이라고도 부름)


### crocross_vaildate
- 사이킷 런에서는 `cross_vaildate()` 함수가 있음 
- 반환 값으로 `fit_time`, `score_time`, `test_score`를 키로 가진 딕셔너리를 반환 
- `fit_time` : 훈련 시간
- `score_time` : 검증 시간 
- `test_score` : 검증 폴드의 점수 -> 평균내서 점수를 구함

In [6]:
from sklearn.model_selection import cross_validate
# 교차 검증
scores = cross_validate(dt, train_input, train_target)
print("훈련 세트 점수 : ", scores)

## 평균 점수 구하기 
import numpy as np
print("훈련 세트 점수 평균 : ", np.mean(scores['test_score']))

훈련 세트 점수 :  {'fit_time': array([0.0210681 , 0.0165379 , 0.02126002, 0.01604962, 0.01053166]), 'score_time': array([0.00551987, 0.00250983, 0.00200939, 0.00400758, 0.00300479]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
훈련 세트 점수 평균 :  0.855300214703487


- 단, `cross_validate`는 훈련 세트를 섞어 폴드를 나누지 않음 
  - train_test_split으로 데이터를 섞은 후 했기 때문에 섞을 필요 없음 
  - 교차 검증을 할 때 훈련 세트를 섞으려면 `분할기`를 사용해야 함 -> StratifiedKFold 를 이용

In [7]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print("훈련 세트 점수 평균 : ", np.mean(scores['test_score']))

훈련 세트 점수 평균 :  0.855300214703487


- 10 폴드 검증

In [8]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print("훈련 세트 점수 평균 : ", np.mean(scores['test_score']))

훈련 세트 점수 평균 :  0.8574181117533719


# 2. 하이퍼 파라미터 튜닝 

## 1) 하이퍼 파라미터 
- `모델 파라미터` : 머신러닝 모델이 학습하는 파라미터 
- `하이퍼 파라미터` : 모델을 학습하기 전에 설정하는 파라미터
  - 사용자가 지정해야만 하는 파라미터 , 학습하지 못하는 파라미터 

## 2) 하이퍼 파리미터 튜닝 
- `하이퍼 파라미터 튜닝` : 하이퍼 파라미터를 조정하는 과정
  - 먼저, 라이브러리 기본값 사용 
- 그 다음 검증세트의 점수나 교차 검증을 통해 매개변수를 조정 



## 3) 그리드 서치 
- 하이퍼 파라미터 조합을 찾기 위한 도구 
- 하이퍼 파라미터 탐색 및 교차 점증을 한 번에 수행 
    - `cross_vaildate()`할 필요 없음 
- `min_impurity_decrease` 매개 변수의 최적값 찾기 
  - min_impurity_decrease 값을 5개 값으로 시도 
- 5 폴드 교차 검증 수행 = > 총 25번 수행 
- `n_jobs` : 병렬 실행에 사용할 CPU 코어 수 지정 
  - `-1` : 모든 코어 사용

-`best_estimator_` : 최적의 값 

In [ ]:
from sklearn.model_selection import GridSearchCV
### 하이퍼파라미터 튜닝을 위한 그리드 서치
### min_impurity_decrease 5개로 설정 
params = {"min_impurity_decrease":[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

### 모델 학습 
gs.fit(train_input, train_target)

### 최적의 값
dt = gs.best_estimator_

### 최적의 값으로 구한 훈련 세트 점수
print(dt.score(train_input, train_target))

0.9615162593804117


In [13]:
### 5번의 교차 검증 으로 얻은 점수 > 1번째가 가장 큼 
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


## 4) 그리드 서치를 이용한 결정 트리 모델
- 3가지 하이퍼 파라미터를 튜닝 
- `min_impurity_decrease` = 노드를 분할하기 위한 불순도 감수의 최소량 
- `max_depth` = 트리의 최대 깊이
- `min_samples_split` = 노드를 분할하기 위한 최소 샘플 수

In [18]:
params = {
    ### 0.0001부터 0.001이 될 때까지 0.0001씩 증가
    "min_impurity_decrease": np.arange(0.0001,0.001,0.0001),
    ### max_depth 5부터 20까지 1씩 증가 (정수만)
    "max_depth": range(5,20,1),
    ### min_samples_leaf 1부터 50까지 1씩 증가 (정수만)
    "min_samples_split": range(2,100,10),
}

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
### 모델 학습
gs.fit(train_input, train_target)
### 최적의 값
dt = gs.best_estimator_
print("최적의 값 : ", dt)

### 최상의 교차 검증 점수 
print("최상의 교차 검증 점수 : ",np.max(gs.cv_results_['mean_test_score']))

최적의 값 :  DecisionTreeClassifier(max_depth=14, min_impurity_decrease=np.float64(0.0004),
                       min_samples_split=12, random_state=42)
최상의 교차 검증 점수 :  0.8683865773302731


## 5) 랜덤 서치 
- 매개 변수의 값의 범위 혹은 간격을 미리 정하기 힘듦 > 랜덤 서치 사용
- 매개 변수의 값을 수치가 아닌 매개변수를 샘플링 할 수 있는 확률 분포를 전달 

- from scipy.stats import uniform, randint : 주어진 범위에서 고르게 > 균등 분포에서 샘플링

In [22]:
from scipy.stats import uniform, randint

## 0~10 범위
rgen = randint(0, 10)
## 10개 추출 
rgen.rvs(10)


array([6, 8, 0, 2, 8, 4, 8, 4, 9, 0])

- 1000개를 샘플링 해서 숫자를 카운트 

In [24]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([103, 103, 103, 108, 102, 106,  89,  95, 103,  88]))

In [26]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.56383485, 0.64642989, 0.48567805, 0.45196288, 0.2129066 ,
       0.29284786, 0.59774333, 0.99229527, 0.78197316, 0.62651971])

## 6) 랜덤 서치를 이용한 튜닝 

In [28]:
params = {
    "min_impurity_decrease": uniform(0.0001, 0.001),
    "max_depth": randint(20, 50),
    "min_samples_split": randint(2,25),
    "min_samples_leaf": randint(1, 25)
}

In [29]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)

### 모델 학습
rs.fit(train_input, train_target)

### 최적의 값
dt = rs.best_estimator_
print("최적의 값 : ", dt)

### 최상의 교차 검증 점수
print("최상의 교차 검증 점수 : ", np.max(rs.cv_results_['mean_test_score']))

### 최적 성능 
print("최적 성능 : ", dt.score(test_input, test_target))

최적의 값 :  DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=np.float64(0.00034102546602601173),
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)
최상의 교차 검증 점수 :  0.8695428296438884
최적 성능 :  0.86
